<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/DiffraxPID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diffrax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.8/159.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 16.3 MB/s eta 0:00:00


In [248]:
from diffrax import diffeqsolve, ODETerm, Dopri5, SaveAt, Kvaerno5, PIDController
import jax.numpy as jnp
import jax
from plotly.subplots import make_subplots
from scipy.optimize import minimize
jax.config.update("jax_enable_x64", True)

In [239]:
a=0.01
A = 1.
qin_initial_ss = 0.1
sq2g = jnp.sqrt(2*9.81)
h10 = h20=qin_initial_ss**2/(a**2)/(2*9.81)
loss0 = 0.
eint0 = 0.
hsp = 1.1*h20

def dhdt(t, hvec, args):
    h1,h2,eint,loss = hvec
    Kc, taui = args
    qin=qin_initial_ss
    e = hsp - h2
    qcontrol= Kc*(e+eint/taui)
    q12 = a*sq2g*(h1**0.5)
    q2 = a*sq2g*(h2**0.5)
    return jnp.array( [(qin + qcontrol - q12)/A, (q12 - q2)/A, e, jnp.abs(e)])


dhdt_term = ODETerm(dhdt)
solver = Kvaerno5()
stepsize=PIDController(rtol=1e-5, atol=1e-8)
def loss(args):
    return diffeqsolve(dhdt_term, solver, t0=0, t1=tend, dt0=0.1, y0=jnp.array([h10, h20, eint0, loss0]), args=args,
                       stepsize_controller=stepsize).ys[0][-1]

loss_jit=jax.jit(loss)
grad_jit=jax.jit(jax.grad(loss))

In [257]:
tend=1000.
args=jnp.array([ 4.553e-01, 5.775e+02])
tplot=jnp.linspace(0,tend,500)
sol=diffeqsolve(dhdt_term, solver, t0=0, t1=tend, dt0=0.1, y0=jnp.array([h10, h20, eint0, loss0]), args=args, saveat=SaveAt(ts=tplot),
                stepsize_controller=stepsize)

In [258]:
fig=make_subplots()
fig.add_scatter(x=sol.ts,y=sol.ys[:,0],mode='lines')
fig.add_scatter(x=sol.ts,y=sol.ys[:,1],mode='lines')
fig.update_layout(width=600,height=600,template='plotly_dark')

In [259]:
loss_jit(args)

Array(40.18685889, dtype=float64, weak_type=True)

In [260]:
grad_jit(args)

Array([-0.04237762,  0.00039427], dtype=float64)

In [261]:
minimize(loss_jit, args, jac=grad_jit, bounds=[(1e-5,0.5),(100.,1e6)], method='SLSQP')

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 39.97493361387476
       x: [ 4.993e-01  5.775e+02]
     nit: 4
     jac: [-8.768e+00 -2.151e-04]
    nfev: 13
    njev: 4